In [15]:
%reset -f
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet201
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import pandas as pd
import cv2, os, os, git, glob, random
import numpy as np
from insectrec.utils import get_dataset, train_generator, valid_generator, augment_trainset
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
seed = 42
np.random.seed(seed)

repo = git.Repo('.', search_parent_directories=True)
created_data_path = f'{repo.working_tree_dir}/insectrec/created_data'
path_impy_crops_export = f'{created_data_path}/impy_crops_export/'
path_images_augmented = f'{created_data_path}/images_augmented/'

if not os.path.isdir(path_images_augmented):
    os.mkdir(path)
    
clean = True
if clean:
    os.system(f'rm -rf {path_images_augmented}*')

epochs = 100
batch_size = 32
monitor='val_loss'
es_patience=7
rlr_patience=4
img_dim = 90
modelname = 'newfuji'

In [16]:
# Creating le for encoding labels
le = LabelEncoder()

# Creating dataframe with all the original data (x: filenames, textlabels, y: nummerical labels)
df_orig = pd.DataFrame()
df_orig['x'] = pd.Series(glob.glob(f"{path_impy_crops_export}/*/*.jpg"))
df_orig['y_text'] = df_orig['x'].apply(lambda x: x.split('/')[-2])
df_orig['y'] = le.fit_transform(df_orig.y_text)

# Splitting into train/val/test
X_train, X_test, y_train, y_test = train_test_split(df_orig.x, df_orig.y, test_size=0.2, random_state=seed, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=seed, shuffle=True)

In [17]:
augment_trainset(X_train=X_train, y_train=y_train, 
                 aug_imgs_path=path_images_augmented, 
                 nb_batches=100, 
                 batch_size=500)

 Reading image data and assigning labels...
Normalizing data by dividing by 255.
Creating an ImageDataGenerator.
Creating directories for each class.
Fitting data generator on data.
Expanding dataset by using generator's flow method on data/labels.
using batch_size of 500
Finished augmentation in 100 batches of 500


In [18]:
# Gathering info on augmented X_train data
df_aug = pd.DataFrame()
df_aug['x'] = pd.Series(glob.glob(f"{path_images_augmented}/*/*.jpg"))
df_aug['textlabels'] = df_aug['x'].apply(lambda x: x.split('/')[-2])
df_aug['y'] = le.fit_transform(df_aug.textlabels)

In [19]:
X_train_aug = df_aug.x.tolist()
y_train_aug = df_aug.y.tolist()

c = list(zip(X_train_aug, y_train_aug))
random.shuffle(c)
X_train_aug, y_train_aug = zip(*c)
X_train_aug = list(X_train_aug)
y_train_aug = list(y_train_aug)
X_val = X_val.tolist()
y_val = y_val.tolist()
X_test = X_test.tolist()
y_test = y_test.tolist()

In [20]:
log_dir = './insectrec/created_data/logs/'
top_weights_path = f'./insectrec/created_data/weights/model_{modelname}_{img_dim}.h5'

callbacks_list = [ModelCheckpoint(monitor =  monitor,
                                  filepath =  top_weights_path,
                                  save_best_only = False,
                                  save_weights_only = False,
                                  verbose = 1),
                  EarlyStopping(monitor =  monitor,
                                patience =  es_patience,
                                verbose = 1),
                  ReduceLROnPlateau(monitor =  monitor,
                                    factor = 0.1,
                                    patience =  rlr_patience,
                                    verbose = 1),
                # CSVLogger(filename =  logfile),
                  TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)]

## TRAINING FROM SCRATCH

In [21]:
base_model = DenseNet121(include_top=True, weights=None, 
                        input_shape=(img_dim,img_dim,3))
x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(6, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [22]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 90, 90, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 96, 96, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 45, 45, 64)   9408        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 45, 45, 64)   256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [23]:
# from tensorflow.keras.optimizers import SGD
# opt = SGD(lr=0.001)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
# model.summary()

In [25]:
import math
FH = model.fit_generator(train_generator(X_train_aug, y_train_aug, batch_size=batch_size, nb_classes=6, img_dim=img_dim), 
                         validation_data=valid_generator(X_val, y_val, batch_size=batch_size, nb_classes=6, img_dim=img_dim), 
                         steps_per_epoch=int(math.ceil(float(len(X_train_aug)) / float(batch_size))), 
                         validation_steps=int(math.ceil(float(len(X_val))/float(batch_size))), 
                         epochs=epochs, 
                         verbose=1, 
                         callbacks=callbacks_list, 
                         shuffle=True)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1520 steps, validate for 33 steps
Epoch 1/100
1519/1520 [============================>.] - ETA: 1s - loss: 1.6132 - accuracy: 0.3230
Epoch 00001: saving model to ./insectrec/created_data/weights/model_newfuji_90.h5
1520/1520 [==============================] - 1914s 1s/step - loss: 1.6132 - accuracy: 0.3230 - val_loss: 1.5591 - val_accuracy: 0.3246
Epoch 2/100
1519/1520 [============================>.] - ETA: 1s - loss: 1.4962 - accuracy: 0.4133
Epoch 00002: saving model to ./insectrec/created_data/weights/model_newfuji_90.h5
1520/1520 [==============================] - 1900s 1s/step - loss: 1.4964 - accuracy: 0.4133 - val_loss: 1.3907 - val_accuracy: 0.5263
Epoch 3/100
1519/1520 [============================>.] - ETA: 1s - loss: 1.4042 - accuracy: 0.4597
Epoch 00003: saving model to ./insectrec/created_data/weights/model_newfuji_90.h5
1520/1520 [==============================] - 1900s 1s/step - loss: 1.4043 - accuracy: 0.4596 

KeyboardInterrupt: 

In [ ]:
def common(a,b): 
    c = [value for value in a if value in b] 
    return c

common(X_test, X_train)

In [ ]:
len(X_train)

## FINE TUNING

In [ ]:
# base_model = DenseNet121(include_top=False, weights='imagenet', 
#                          input_shape=(img_dim,img_dim,3))
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# predictions = Dense(6, activation='softmax')(x)

# model = Model(inputs=base_model.input, outputs=predictions)

# # let's visualize layer names and layer indices to see how many layers
# # we should freeze:
# #for i, layer in enumerate(base_model.layers):
# #   print(i, layer.name)

In [ ]:
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# # train the model on the new data for a few epochs
# import math
# FH = model.fit_generator(train_generator(X_train_aug, y_train, batch_size=batch_size, nb_classes=6, img_dim=img_dim),
#                          validation_data=valid_generator(X_val, y_val, batch_size=batch_size, nb_classes=6, img_dim=img_dim), 
#                          steps_per_epoch=int(math.ceil(float(len(X_train_aug)) / float(batch_size))),
#                          validation_steps=int(math.ceil(float(len(X_val))/float(batch_size))),
#                          epochs=epochs, 
#                          verbose=1, 
#                          callbacks=callbacks_list)

In [ ]:
# # we chose to train the top 2 inception blocks, i.e. we will freeze
# # the first 249 layers and unfreeze the rest:
# for layer in model.layers[:425]:
#    layer.trainable = False
# for layer in model.layers[425:]:
#    layer.trainable = True

# # we need to recompile the model for these modifications to take effect
# # we use SGD with a low learning rate
# from tensorflow.keras.optimizers import SGD
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # we train our model again (this time fine-tuning the top 2 inception blocks
# # alongside the top Dense layers
# FH2 = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch_size),
#     validation_data=(testX, testY), steps_per_epoch=len(trainX) // batch_size,
#     epochs=epochs, verbose=1, callbacks=callbacks_list)

## EXTRACTING FEATURES